In [45]:
import numpy as np                   
import pandas as pd

In [46]:
import pytesseract
from PIL import Image

# Set Tesseract Path
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


In [47]:
import pytesseract

try:
    print("Tesseract Version:", pytesseract.get_tesseract_version())
except:
    print("Error: Tesseract is not installed or not configured properly.")


Tesseract Version: 5.5.0.20241111


In [48]:
import os
from PIL import Image
import pytesseract

# Set Tesseract Path
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Correct image path
image_path = r"C:\Github\Banking-Documentation-Validation-and-account-creation\data\aadhaar_kyc_dataset\images\aadhaar_0.jpg"

# Check if the file exists
if not os.path.exists(image_path):
    print(f"❌ Error: Image file not found at {image_path}")
else:
    # Load Image and Extract Text
    image = Image.open(image_path)
    extracted_text = pytesseract.image_to_string(image)
    
    print("✅ Extracted Text:\n", extracted_text)


✅ Extracted Text:
 Name: Meera Sule

DOB: 24/09/2023

Gender: Other

Aadhaar No: 941524424120

Address: 22/86, Sachar Ganj, Hapur-392885




In [49]:
import re

def is_aadhaar_card(text):
    """
    Function to check if extracted text belongs to an Aadhaar Card.
    """
    # Aadhaar number pattern (12-digit number)
    aadhaar_pattern = r'\b\d{12}\b'

    # Keywords commonly found in Aadhaar
    keywords = ["Aadhaar", "Aadhaar No", "DOB", "Gender", "Address"]

    # Check if any keyword is present
    keyword_match = any(keyword.lower() in text.lower() for keyword in keywords)

    # Check if a valid 12-digit Aadhaar number is present
    aadhaar_match = re.search(aadhaar_pattern, text)

    if keyword_match and aadhaar_match:
        return True  # It's an Aadhaar Card
    return False  # Not an Aadhaar Card


In [50]:
# Check if the extracted text belongs to an Aadhaar Card
if is_aadhaar_card(extracted_text):
    print("✅ The document is an Aadhaar Card.")
else:
    print("❌ The document is NOT an Aadhaar Card.")


✅ The document is an Aadhaar Card.


In [51]:
def extract_aadhaar_details(text):
    """
    Function to extract Name, DOB, Gender, Aadhaar Number, and Address from Aadhaar OCR text.
    """
    details = {"Name": None, "DOB": None, "Gender": None, "Aadhaar Number": None, "Address": None}

    # Extract Name
    name_match = re.search(r"Name[:\s]+([A-Za-z\s]+)", text)
    if name_match:
        details["Name"] = name_match.group(1).strip()

    # Extract Date of Birth (DOB) - Ensure it’s not repeated
    dob_match = re.search(r"DOB[:\s]+(\d{2}/\d{2}/\d{4})", text)
    if dob_match:
        details["DOB"] = dob_match.group(1)

    # Extract Gender
    gender_match = re.search(r"Gender[:\s]+(Male|Female|Other)", text, re.IGNORECASE)
    if gender_match:
        details["Gender"] = gender_match.group(1).capitalize()

    # Extract Aadhaar Number
    aadhaar_match = re.search(r"\b\d{12}\b", text)
    if aadhaar_match:
        details["Aadhaar Number"] = aadhaar_match.group(0)

    # Extract Address
    address_match = re.search(r"Address[:\s]+(.+)", text)
    if address_match:
        details["Address"] = address_match.group(1).strip()

    return details


In [52]:
# Extract structured details from Aadhaar OCR text
aadhaar_details = extract_aadhaar_details(extracted_text)

# Print extracted details
print("\n✅ Extracted Aadhaar Details:")
for key, value in aadhaar_details.items():
    print(f"{key}: {value}")



✅ Extracted Aadhaar Details:
Name: Meera Sule

DOB
DOB: 24/09/2023
Gender: Other
Aadhaar Number: 941524424120
Address: 22/86, Sachar Ganj, Hapur-392885


In [53]:
import json

# Extract details again
aadhaar_details = extract_aadhaar_details(extracted_text)

# Convert to JSON format
aadhaar_json = json.dumps(aadhaar_details, indent=4)

print("\n✅ Aadhaar Details in JSON Format:")
print(aadhaar_json)



✅ Aadhaar Details in JSON Format:
{
    "Name": "Meera Sule\n\nDOB",
    "DOB": "24/09/2023",
    "Gender": "Other",
    "Aadhaar Number": "941524424120",
    "Address": "22/86, Sachar Ganj, Hapur-392885"
}


In [ ]:
import os
import re
from PIL import Image
import pytesseract

# Function to extract details from OCR text
def extract_aadhaar_details(text):
    details = {"Name": None, "DOB": None, "Gender": None, "Aadhaar Number": None, "Address": None}
    name_match = re.search(r"Name[:\s]+([A-Za-z\s]+(?:[A-Za-z\s]+)*)", text)
    if name_match:
        details["Name"] = name_match.group(1).strip().replace("\n", " ")

    dob_match = re.search(r"DOB[:\s]+(\d{2}/\d{2}/\d{4})", text)
    if dob_match:
        details["DOB"] = dob_match.group(1)

    gender_match = re.search(r"Gender[:\s]+(Male|Female|Other)", text, re.IGNORECASE)
    if gender_match:
        details["Gender"] = gender_match.group(1).capitalize()

    aadhaar_match = re.search(r"\b\d{12}\b", text)
    if aadhaar_match:
        details["Aadhaar Number"] = aadhaar_match.group(0)

    address_match = re.search(r"Address[:\s]+(.+)", text)
    if address_match:
        details["Address"] = address_match.group(1).strip()

    return details

# Function to read label from a .txt file
def read_aadhaar_label(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    return text.strip().replace("\n", " ")

# Function to compare OCR details with the label
def compare_details(ocr_details, label_details):
    comparison = {}
    for key in ocr_details:
        label_value = re.search(f"{key}[:\s]+([A-Za-z0-9\s,]+)", label_details)
        if label_value:
            label_value = label_value.group(1).strip()
            comparison[key] = {
                "OCR": ocr_details[key],
                "Label": label_value,
                "Match": ocr_details[key] == label_value
            }
    return comparison

# Main processing function for all images and labels
def process_aadhaar_data(image_folder, label_folder):
    images = os.listdir(image_folder)  # List all image files
    comparisons = []

    for image_name in images:
        if image_name.endswith(".jpg") or image_name.endswith(".png"):
            # Get the image path
            image_path = os.path.join(image_folder, image_name)
            
            # Get corresponding label file
            label_file_name = image_name.split('.')[0] + ".txt"
            label_path = os.path.join(label_folder, label_file_name)
            
            # Read the label file
            label_details = read_aadhaar_label(label_path)
            
            # Perform OCR on the image
            image = Image.open(image_path)
            extracted_text = pytesseract.image_to_string(image)
            
            # Extract OCR details
            ocr_details = extract_aadhaar_details(extracted_text)
            
            # Compare OCR details with label details
            comparison = compare_details(ocr_details, label_details)
            comparisons.append(comparison)

            # Print results for each image
            print(f"Results for {image_name}:")
            for field, result in comparison.items():
                print(f"{field}:")
                print(f"  OCR: {result['OCR']}")
                print(f"  Label: {result['Label']}")
                print(f"  Match: {result['Match']}")
                print("\n")
    
    return comparisons

# Run the function
image_folder = r"C:\Github\Banking-Documentation-Validation-and-account-creation\data\aadhaar_kyc_dataset\images"
label_folder = r"C:\Github\Banking-Documentation-Validation-and-account-creation\data\aadhaar_kyc_dataset\labels"
process_aadhaar_data(image_folder, label_folder)


In [56]:
import os
import re
import csv
from PIL import Image
import pytesseract

# Function to extract details from OCR text
def extract_aadhaar_details(text):
    details = {"Name": None, "DOB": None, "Gender": None, "Aadhaar Number": None, "Address": None}
    name_match = re.search(r"Name[:\s]+([A-Za-z\s]+(?:[A-Za-z\s]+)*)", text)
    if name_match:
        details["Name"] = name_match.group(1).strip().replace("\n", " ")

    dob_match = re.search(r"DOB[:\s]+(\d{2}/\d{2}/\d{4})", text)
    if dob_match:
        details["DOB"] = dob_match.group(1)

    gender_match = re.search(r"Gender[:\s]+(Male|Female|Other)", text, re.IGNORECASE)
    if gender_match:
        details["Gender"] = gender_match.group(1).capitalize()

    aadhaar_match = re.search(r"\b\d{12}\b", text)
    if aadhaar_match:
        details["Aadhaar Number"] = aadhaar_match.group(0)

    address_match = re.search(r"Address[:\s]+(.+)", text)
    if address_match:
        details["Address"] = address_match.group(1).strip()

    return details

# Function to read label from a .txt file
def read_aadhaar_label(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    return text.strip().replace("\n", " ")

# Function to compare OCR details with the label
def compare_details(ocr_details, label_details):
    comparison = {}
    for key in ocr_details:
        label_value = re.search(f"{key}[:\s]+([A-Za-z0-9\s,]+)", label_details)
        if label_value:
            label_value = label_value.group(1).strip()
            comparison[key] = {
                "OCR": ocr_details[key],
                "Label": label_value,
                "Match": ocr_details[key] == label_value
            }
    return comparison

# Function to save results to CSV
def save_results_to_csv(results, output_file):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write headers
        writer.writerow(["Image Name", "Field", "OCR", "Label", "Match"])
        for result in results:
            for field, details in result.items():
                writer.writerow([result['image_name'], field, details["OCR"], details["Label"], details["Match"]])

# Main processing function for all images and labels
def process_aadhaar_data(image_folder, label_folder, output_csv):
    images = os.listdir(image_folder)  # List all image files
    comparisons = []

    for image_name in images:
        if image_name.endswith(".jpg") or image_name.endswith(".png"):
            # Get the image path
            image_path = os.path.join(image_folder, image_name)
            
            # Get corresponding label file
            label_file_name = image_name.split('.')[0] + ".txt"
            label_path = os.path.join(label_folder, label_file_name)
            
            # Read the label file
            label_details = read_aadhaar_label(label_path)
            
            # Perform OCR on the image
            image = Image.open(image_path)
            extracted_text = pytesseract.image_to_string(image)
            
            # Extract OCR details
            ocr_details = extract_aadhaar_details(extracted_text)
            
            # Compare OCR details with label details
            comparison = compare_details(ocr_details, label_details)
            
            # Add image name to comparison result
            comparison['image_name'] = image_name
            comparisons.append(comparison)

    # Save results to CSV
    save_results_to_csv(comparisons, output_csv)
    print(f"Results saved to {output_csv}")

# Run the function
image_folder = r"C:\Github\Banking-Documentation-Validation-and-account-creation\data\aadhaar_kyc_dataset\images"
label_folder = r"C:\Github\Banking-Documentation-Validation-and-account-creation\data\aadhaar_kyc_dataset\labels"
output_csv = r"C:\Github\Banking-Documentation-Validation-and-account-creation\data\aadhaar_comparisons.csv"
process_aadhaar_data(image_folder, label_folder, output_csv)


KeyboardInterrupt: 